In [1]:
# Ok so first of all the instructions here (https://cryptopals.com/sets/5/challenges/39) leave a lot out
# And the exercise doesn't work if you follow the instructions specifically... they leave out a lot about the totient function
# I followed Wikipedia instead (https://en.m.wikipedia.org/wiki/RSA_(cryptosystem))


from Crypto.Util.number import getPrime
Prime = lambda : getPrime(1024)

In [3]:
# Using implementations from https://github.com/ricpacca/cryptopals/blob/master/S5C39.py

def GCD(a, b):
    """Computes the greatest common divisor between a and b using the Euclidean algorithm."""
    while b != 0:
        a, b = b, a % b

    return a


def LCM(a, b):
    """Computes the lowest common multiple between a and b using the GCD method."""
    return a // GCD(a, b) * b


def ModInv(a, n):
    """Computes the multiplicative inverse of a modulo n using the extended Euclidean algorithm."""
    t, r = 0, n
    new_t, new_r = 1, a

    while new_r != 0:
        quotient = r // new_r
        t, new_t = new_t, t - quotient * new_t
        r, new_r = new_r, r - quotient * new_r

    if r > 1:
        raise Exception("a is not invertible")
    if t < 0:
        t = t + n

    return t

assert ModInv(17, 3120) == 2753

In [4]:
# Let e be 3
e = 3

In [17]:
# The requirement regarding GCD is specified in Wikipedia, NOT the challenge itself

def GeneratePrimes(e = e):
    while True:
        p, q = Prime(), Prime()
        n = p*q
        et = LCM(p-1, q-1) % n

        if 2 < e < et and GCD(e, et) == 1:
            return p, q, et

p, q, et = GeneratePrimes()

In [18]:
d = ModInv(e, et)
print(d)

assert (d*e) % et == 1

4327016016097887518670804796858228752674491342066273402146279437795173715274408180432671216215513159944939217378525420465700294462808034701633153361785675301344310022006296860681785759987472738282474660074686102962193930463669088554377490370491150767878153110384210993297036291030123443474927005267843606889054504321466887033192930507514919061607665089370766142019970009708832936312190978213507808141865472119822739150567207442769775527284305099574490807257323214674410363952703231154679020557804575800114456558795668752168344256902720699509612911272804030684238541190725155323884964113879932791219071306509681576787


In [49]:
class RSAServer(object):
    def __init__(self, e = 3):
        self.e = e
        p, q, et = GeneratePrimes(e)
        self.p = p
        self.q = q
        self.d = ModInv(e, et)
        self.n = p*q
        
    def GetPubkey(self):
        return self.e, self.n
    
    def Decrypt(self, c):
        m = pow(c, self.d, self.n)
        return m.to_bytes(byteorder='big', length = 1024//8)

class RSAClient(object):
    def __init__(self, e, n):
        self.e = e
        self.N = n
        
    def Encrypt(self, message):
        m = message if type(message) is int else int.from_bytes(message, byteorder = 'big')
        assert m < n
        return pow(m, self.e, self.N)
        

In [50]:
server = RSAServer()
e, n = server.GetPubkey()
client = RSAClient(e = e, n = n)

In [53]:
message = b'Hello, world!'
ciphertext = client.Encrypt(message)
ciphertext

188706042416845720494214359667263833254252079538597449951674686304003868196918785764108580961

In [55]:
server.Decrypt(ciphertext).replace(b'\0', b'')

b'Hello, world!'